In [ ]:
# default_exp core

# MLDaTo

> API details. MLDaTo stand for machine learning on any data in an automated fasion

In [2]:
#hide
#from nbdev.showdoc import *

import pandas as pd
import numpy as np
df=pd.read_csv('D:/Work/R Packages/Testing data/CreditModellingTestCase.csv',sep=';')
df = df.loc[df['default'].notnull()]

# Exploratory data analysis

* Depends: pandas, numpy

In [4]:
def explore_df(df):
    
    """
    A more advanced version of describe for tabular exploratory data analysis. Inlcudes additional information such as,
    missing observations, unique observations, constant feature flagging, all_missing feature flagging, feature types & outlier
    values.

    Parameters
    ----------
    df : pandas df, required, default=NA
        Pandas dataframe object 

    Returns
    -------
    pandas df
        Returns a pandas dataframe object
    
    Usage
    -----
        df = pd.DataFrame({"x1": ["a", "b", "c", "a"], "x2":['x','y','x','x'], "y": [1,1,0,1]})
        eda = explore_df(df=df)
    """
    
    import pandas as pd
    import numpy as np
    
    ft = pd.DataFrame()
    ft['type']=df.dtypes.astype(str)
    ft['feature']=ft.index
    ft['unique']=df.nunique()
    ft['missing']= df.isnull().sum()
    ft['constant']=np.where(ft['unique']==1,1,0)
    ft['all_missing']=np.where(ft['missing']==df.shape[0],1,0)

    numeric = ft.loc[(ft['type'].str.contains('float'))]['feature']
    numeric = numeric.append(ft.loc[(ft['type'].str.contains('int'))]['feature'])
    
    categorical = ft.loc[(ft['type'].str.contains('object'))]['feature']

    # Summary statistics
    lower=df[numeric].quantile(q=0.25)
    upper=df[numeric].quantile(q=0.75)
    ft['min']=df[numeric].min()
    ft['q1']=lower
    ft['median']=df[numeric].median()
    ft['mean']=df[numeric].mean()
    ft['q3']=upper
    ft['max']=df[numeric].max()

    # Caclulate outlier values
    iqr = upper - lower
    lower=lower-(1.5*iqr)
    upper=upper+(1.5*iqr)
    ft['lower_outlier']=lower
    ft['upper_outlier']=upper
    ft['skewness']=df[numeric].skew()
    
    ft['class'] = np.where(ft['type'].str.contains('float'), 'numeric', None)
    ft['class'] = np.where(ft['type'].str.contains('int'), 'numeric', ft['class'])
    ft['class'] = np.where(ft['type'].str.contains('object'), 'categorical', ft['class'])
    ft['class'] = np.where(ft['type'].str.contains('datetime'), 'datetime', ft['class'])
    ft['class'] = np.where(ft['class'].isin(['numeric','integer']) & 
                           (ft['min'] == 0) & 
                           (ft['max'] == 1) & 
                           (ft['unique'] == 2), 'indicator', ft['class'])
        
    ft=ft[['feature','type','class','missing','unique','constant','all_missing','min','q1','median',
         'mean','q3','max','lower_outlier','upper_outlier','skewness']]

    ft=ft.reset_index(drop=True)
    return ft

In [5]:
# TESTS
eda = explore_df(df)

# Format features

* Depends: numpy, pandas

In [6]:
def format_features(df):

    import pandas as pd
    import numpy as np
    
    """
    Applies feature formatting to features to comply with machine learning models. Boolean features are transformed to integer 
    and objects (where applicable) are tried to be formatted as datetime. The resulting feature set should only contain float,
    integer, object and datetime type features.

    Parameters
    ----------
    df : pandas df, required, default=NA
        Pandas dataframe object 

    Returns
    -------
    pandas df
        Returns a pandas dataframe object
    
    Usage
    -----
        df = pd.DataFrame({"x1": ["a", "b", "c", "a"], "x2":['x','y','x','x'], "y": [1,1,0,1]})
        df = format_features(df=df)
    """
    
    # Try and convert relevant datetime columns to datetime format
    df = df.apply(lambda col: pd.to_datetime(col, errors='ignore') 
              if col.dtypes == object 
              else col, 
              axis=0)

    ft = pd.DataFrame()
    ft['from_type'] = df.dtypes.astype(str)
    ft['feature'] = ft.index
    logical = ft.loc[ft['from_type'] == 'bool']['feature'].unique()
    df[logical] = df[logical].astype(int) 
    return df

In [7]:
# TESTS
df = format_features(df)

# Detect features

* TODO: Add in functionality to detect flag features, having nr of uniques = 2 and set values
* Depends: numpy, pandas

In [8]:
def detect_features(df, y = None, id = None):
    
    import numpy as np
    import pandas as pd
    
    """
    Detects features of different types and returns each type in individual series objects. Feature types detected and grouped are
    numeric (float and integer), categorical (object and category), datetime (date) and flags (unique of 2 containing special
    values).

    Parameters
    ----------
    df : pandas df, required, default=NA
        Pandas dataframe object 
    y: str, optional, default=None
        Name of the target feature
    id: str, optional, default=None
        Name of the id feature

    Returns
    -------
    pandas series
        Returns a series for numeric, categorical, datetime and flag features in that order
    
    Usage
    -----
        df = pd.DataFrame({"x1": ["a", "b", "c", "a"], "x2":['x','y','x','x'], "y": [1,1,0,1]})
        numeric, categorical, datetime, flag = detect_features(df)
    """

    ft = pd.DataFrame()
    ft['type'] = df.dtypes.astype(str)
    ft['feature']=ft.index
    ft['unique']=df.nunique()
    ft.reset_index(drop=True, inplace=True)
    numeric = ft.loc[(ft['type'].str.contains('float'))]['feature']
    numeric = numeric.append(ft.loc[(ft['type'].str.contains('int'))]['feature'])

    categorical = ft.loc[(ft['type'].str.contains('object'))]['feature']
    categorical = categorical.append(ft.loc[(ft['type'].str.contains('category'))]['feature'])

    datetime = ft.loc[(ft['type'].str.contains('date'))]['feature']

    if y is not None:
        numeric = numeric.loc[numeric != y]
        categorical = categorical.loc[categorical != y]

    if id is not None:
        numeric = numeric.loc[numeric != id]
        categorical = categorical.loc[categorical != id]

    return numeric, categorical, datetime

In [10]:
# TESTS
numeric, categorical, datetime = detect_features(df)

# Parition data

Parition data into a train, validation and test set either by stratified random sampling or time sensitive partitioning. For time sensitive partitioning, the data is sorted by the time dependent feature and then split according to percentages of the data running acrros time.

In [ ]:
y = 'default'
x = ['account_amount_added_12_24m',
       'account_days_in_dc_12_24m', 'account_days_in_rem_12_24m']
test_percentage = 0.2
valid_percentage = 0.1
time_based_split_feature = None
seed = 1234
max_uniques = 100

from sklearn.model_selection import train_test_split
import pandas as pd

n_test = df.shape[0] * test_percentage
n_valid = df.shape[0] * valid_percentage

if time_based_split_feature is not None:
    df = df.sort_values(time_based_split_feature)
    test = df.tail(round(n_test))
    
    train = df.iloc[ : df.shape[0] - round(n_test)]
    valid = train.tail(round(n_valid))
    
    train = train.iloc[ : train.shape[0] - round(n_valid)]

if y is not None:
    
    # Classification outcome
    if df[y].nunique() <= max_uniques:
        
        df[y] = df[y].astype('category').cat.codes # Transform target feature
        
        x_train, x_test, y_train, y_test = train_test_split(df[x], df[y], stratify=df[y], 
                                                        test_size=test_percentage, random_state=seed)
        
        train = x_train.copy()
        train[y] = y_train
        
        test = x_test.copy()
        test[y] = y_test
        
        # Caluclate new validation split   
        valid_percentage = n_valid / x_train.shape[0]

        x_train, x_valid, y_train, y_valid = train_test_split(train[x], train[y], stratify=train[y], 
                                                            test_size=valid_percentage, random_state=seed)

        train = x_train.copy()
        train[y] = y_train

        valid = x_valid.copy()
        valid[y] = y_valid
    
    # Regression
    if df[y].nunique() > max_uniques:
        
        x_train, x_test, y_train, y_test = train_test_split(df[x], df[y], test_size=test_percentage, random_state=seed)
        
        train = x_train.copy()
        train[y] = y_train
        
        test = x_test.copy()
        test[y] = y_test
        
        # Caluclate new validation split   
        valid_percentage = n_valid / x_train.shape[0]

        x_train, x_valid, y_train, y_valid = train_test_split(train[x], train[y], test_size=valid_percentage, random_state=seed)

        train = x_train.copy()
        train[y] = y_train

        valid = x_valid.copy()
        valid[y] = y_valid